In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [21]:
df_meta = pd.read_csv('../data/modeling/BUILDING/feature_data_BUILDING_2.csv', sep='|')

In [ ]:
def get_more_image_info(ID):
    url = ''
    # Get Picture Num Favorites
    r = requests

In [22]:
df_meta.head()

,owner,id,red_bin1_nbins255,red_bin2_nbins255,red_bin3_nbins255,red_bin4_nbins255,red_bin5_nbins255,red_bin6_nbins255,red_bin7_nbins255,red_bin8_nbins255,...,bright_ctr2_numctrs3,bright_ctr3_numctrs3,bright_ctr1_numctrs8,bright_ctr2_numctrs8,bright_ctr3_numctrs8,bright_ctr4_numctrs8,bright_ctr5_numctrs8,bright_ctr6_numctrs8,bright_ctr7_numctrs8,bright_ctr8_numctrs8
0,100477852@N05,22334842890,0.049714,0.007710,0.004919,0.004460,0.004275,0.004307,0.004183,0.004269,...,180.156076,121.404213,65.621626,45.136088,156.713228,180.494681,129.341803,98.223681,203.314016,16.978573
1,10075702@N00,27207215286,0.020037,0.010178,0.009991,0.009921,0.009359,0.008085,0.006747,0.005709,...,119.609792,38.601534,184.079172,158.257146,106.486092,132.946391,214.015711,76.576349,13.408659,45.902885


In [35]:
df = df_meta[['owner', 'id']]
df

,owner,id
0,100477852@N05,22334842890
1,10075702@N00,27207215286


In [39]:
def get_flickr_keys(filename='../flickr.keys'):
    """Get the Ficker user and app keys from the private .keys file."""
    flickr_keys = {}
    with open(filename) as f_open:
        key_list = f_open.read().split('\n')[0:2]
    for key in key_list:
        pair = key.split(":")
        flickr_keys[pair[0]] = pair[1]
    return flickr_keys

In [102]:
def get_user_data(user_id):
    """
    Get data from Flickr API about the user.

    INPUTS:
    user_id | A user NSID number. (string)

    OUTPUTS:
    user_stats | A 1D Numpy Array of the following user statistics:
                 is_pro (int bool)
                 can_buy_pro (int bool)
                 gender (string)
                 total_views (int)
    """
    url = 'https://api.flickr.com/services/rest/'
    api_keys = get_flickr_keys()
    # Method: flickr.people.getInfo
    params = {'method': 'flickr.people.getInfo',
              'api_key': api_keys['Key'],
              'user_id': user_id
              } 
    soup = BeautifulSoup(requests.get(url, params=params).content, 'lxml')
    is_pro = int(soup.person.get('ispro'))
    can_buy_pro = int(soup.person.get('can_buy_pro'))
    total_views = int(soup.count.contents[0])
    return pd.Series(data=[is_pro, can_buy_pro, total_views])

def get_image_data(image_id):
    """
    Get data from Flickr API about the image.

    INPUTS:
    image_id | An image ID number. (string)

    OUTPUTS:
    image_stats | A 1D Numpy Array of the following user statistics:
                 is_pro (int bool)
                 can_buy_pro (int bool)
                 gender (string)
                 total_views (int)
    """
    url = 'https://api.flickr.com/services/rest/'
    api_keys = get_flickr_keys()
    params = {'method': None,
              'api_key': api_keys['Key'],
              'photo_id': image_id
              }
    # Method: flickr.photos.comments.getList
    params['method'] = 'flickr.photos.comments.getList'
    soup = BeautifulSoup(requests.get(url, params=params).content, 'lxml')
    image_ncomments = len(soup.findAll('comment'))
    # Method: flickr.photos.getFavorites
    params['method'] = 'flickr.photos.getFavorites'
    soup = BeautifulSoup(requests.get(url, params=params).content, 'lxml')
    image_nfavs = soup.photo.get('total')
    # Method: flickr.photos.getAllContexts
    params['method'] = 'flickr.photos.getAllContexts'
    soup = BeautifulSoup(requests.get(url, params=params).content, 'lxml')
    image_nsets = len(soup.findAll('set'))
    image_npools = len(soup.findAll('pool'))
    return pd.Series(data=[image_ncomments, image_nfavs, image_nsets, image_npools])

In [78]:
new = df['owner'].apply(get_user_data)

In [80]:
new.columns = ['user_is_pro', 'user_can_buy_pro', 'user_total_views']

In [97]:
pd.concat((df, new), axis=1)

,owner,id,user_is_pro,user_can_buy_pro,user_total_views
0,100477852@N05,22334842890,0,0,417
1,10075702@N00,27207215286,1,0,3379


In [103]:
get_image_data('22094071913')

0     168
1    3866
2       0
3      13
dtype: object